In [1]:
import torch
import numpy as np
import pandas as pd
from itertools import cycle
from torch_geometric.data import Data
from src.data_preparation import DataPreprocessor
from src.graph_data_loader import graph_loader

### Generating Train Test Split Excel Files

In [2]:
!python src/generate_splits.py

### Graph Dataloader Example

In [2]:
train_graph, test_graph = graph_loader(split_type='shuffled')

Reading train data from  data/splits/shuffled/train.xlsx
Reading test data from  data/splits/shuffled/test.xlsx


Data(x=[3037, 20], edge_index=[2, 4288])

#### Train Graph Attributes

In [3]:
print("Number of nodes: ", train_graph.num_nodes)
print("Number of features per node: ", train_graph.num_node_features)
print("Number of edges: ", train_graph.num_edges)
print("Number of edge attributes: ", train_graph.num_edge_features)
print("Undirected: ", train_graph.is_undirected())
print("Isolated Nodes: ", train_graph.has_isolated_nodes())
print("Self Loops: ", train_graph.has_self_loops())

Number of nodes:  7085
Number of features per node:  20
Number of edges:  22726
Number of edge attributes:  0
Undirected:  True
Isolated Nodes:  True
Self Loops:  False


#### Test Graph Attributes

In [4]:
print("Number of nodes: ", test_graph.num_nodes)
print("Number of features per node: ", test_graph.num_node_features)
print("Number of edges: ", test_graph.num_edges)
print("Number of edge attributes: ", test_graph.num_edge_features)
print("Undirected: ", test_graph.is_undirected())
print("Isolated Nodes: ", test_graph.has_isolated_nodes())
print("Self Loops: ", test_graph.has_self_loops())

Number of nodes:  3037
Number of features per node:  20
Number of edges:  4288
Number of edge attributes:  0
Undirected:  True
Isolated Nodes:  True
Self Loops:  False
